# Decision Tree Model on the Titanic Dataset

Click [here](https://forem.julialang.org/mlj/julia-boards-the-titanic-1ne8) for the blog post.

## Package Installation

We start by creating a new Julia package environment called `titanic`, for tracking versions of the packages we will need. Do this by typing these commands at the `julia>` prompt, with carriage returns added at the end of each line:

In [1]:
# Using Pkg we activate our 'titanic' environment, stored in the shared area.
using Pkg 
Pkg.activate("titanic", shared=true)

  Activating project at `C:\Users\luked\.julia\environments\titanic`


To add the packages we need to your environment, enter the `]` character at the `julia>` prompt, to change it to `(titanic) pkg>`. Then enter: `add MLJ DataFrames BetaML` in the console. It may take a few minutes for these packages to be installed and "precompiled".

Next time you want to use exactly the same combination of packages in a new Julia session, you can skip the `add` command and instead just enter the two lines above them.


When the `(titanic) pkg>` prompt returns, enter `status` to see the package versions that were installed. Here's what each package does:
 - MLJ (machine learning toolbox): provides a common interface for interacting with models provided by different packages, and for automating common model-generic tasks, such as hyperparameter optimization demonstrated at the end of this.
 - DataFrames: Allows you to manipulate tabular data that fits into memory.
 - BetaML: Provides the core decision algorithm we will be building for Titanic prediction.
 
Now return to the notebook.

## Establishing correct data representation

In [2]:
# We establish the correct data representation
using MLJ  # MLJ is a machine learning toolbox
import DataFrames as DF  # we store our data in dataframes

After entering the first line above we are ready to use any function in MLJ's documentation as it appears there. After the second, we can use functions from DataFrames, but must qualify the function names with a prefix `DF.`, as we'll see later.

In MLJ, and some other statistics packages, a "scientific type" or scitype indicates how MLJ will interpret data (as opposed to how it is represented on your machine). Model data requirements are articulated using scitypes, which allows you to focus on what your data represents in the real world, instead of how it is stored on your computer.

We'll grab our Titanic data set from OpenML, a platform for sharing machine learning datasets and workflows. The second line below converts the downloaded data into a dataframe and the third gets summary statistics forthe features in our dataset:

In [3]:
table = OpenML.load(42638)  # load the titanic data
df = DF.DataFrame(table)  # assign it to a DataFrame
DF.describe(df)  # show a brief description of the dataframe df

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,pclass,,1,,3,0,"CategoricalValue{String, UInt32}"
2,sex,,female,,male,0,"CategoricalValue{String, UInt32}"
3,age,29.7589,0.42,30.0,80.0,0,Float64
4,sibsp,0.523008,0.0,0.0,8.0,0,Float64
5,fare,32.2042,0.0,14.4542,512.329,0,Float64
6,cabin,,E31,,C148,687,"Union{Missing, CategoricalValue{String, UInt32}}"
7,embarked,,C,,S,2,"Union{Missing, CategoricalValue{String, UInt32}}"
8,survived,,0,,1,0,"CategoricalValue{String, UInt32}"


In particular, we see that `cabin` has a lot of missing values, and we'll shortly drop it for simplicity.

To get a summary of feature scitypes, we use `schema`:

In [4]:
schema(df)  # gets the scitypes and types of each feature

┌──────────┬─────────────────────────────────┬──────────────────────────────────
│ names    │ scitypes                        │ types                           ⋯
├──────────┼─────────────────────────────────┼──────────────────────────────────
│ pclass   │ Multiclass{3}                   │ CategoricalValue{String, UInt32 ⋯
│ sex      │ Multiclass{2}                   │ CategoricalValue{String, UInt32 ⋯
│ age      │ Continuous                      │ Float64                         ⋯
│ sibsp    │ Continuous                      │ Float64                         ⋯
│ fare     │ Continuous                      │ Float64                         ⋯
│ cabin    │ Union{Missing, Multiclass{186}} │ Union{Missing, CategoricalValue ⋯
│ embarked │ Union{Missing, Multiclass{3}}   │ Union{Missing, CategoricalValue ⋯
│ survived │ Multiclass{2}                   │ CategoricalValue{String, UInt32 ⋯
└──────────┴─────────────────────────────────┴──────────────────────────────────
                            

Now `sibsp` represents the number of siblings/spouses, which is not a continuous variable. So we fix that like this:

In [5]:
coerce!(df, :sibsp => Count)  # change the scitype of sibsp to Count, ! replaces it

Row,pclass,sex,age,sibsp,fare,cabin,embarked,survived
,Cat…,Cat…,Float64,Int64,Float64,Cat…?,Cat…?,Cat…
1,3,male,22.0,1,7.25,missing,S,0
2,1,female,38.0,1,71.2833,C85,C,1
3,3,female,26.0,0,7.925,missing,S,1
4,1,female,35.0,1,53.1,C123,S,1
5,3,male,35.0,0,8.05,missing,S,0
6,3,male,30.0,0,8.4583,missing,Q,0
7,1,male,54.0,0,51.8625,E46,S,0
8,3,male,2.0,3,21.075,missing,S,0
9,3,female,27.0,0,11.1333,missing,S,1


Call `schema(df)` again, to check a successful change.

In [6]:
schema(df)  # check scitypes again

┌──────────┬─────────────────────────────────┬──────────────────────────────────
│ names    │ scitypes                        │ types                           ⋯
├──────────┼─────────────────────────────────┼──────────────────────────────────
│ pclass   │ Multiclass{3}                   │ CategoricalValue{String, UInt32 ⋯
│ sex      │ Multiclass{2}                   │ CategoricalValue{String, UInt32 ⋯
│ age      │ Continuous                      │ Float64                         ⋯
│ sibsp    │ Count                           │ Int64                           ⋯
│ fare     │ Continuous                      │ Float64                         ⋯
│ cabin    │ Union{Missing, Multiclass{186}} │ Union{Missing, CategoricalValue ⋯
│ embarked │ Union{Missing, Multiclass{3}}   │ Union{Missing, CategoricalValue ⋯
│ survived │ Multiclass{2}                   │ CategoricalValue{String, UInt32 ⋯
└──────────┴─────────────────────────────────┴──────────────────────────────────
                            

## Splitting into train and test sets

To objectively evaluate the performance of our final model, we split off 30% of our data into a holdout set, called `df_test`, which will not used for training. You can check the number of observations in each set with `DF.nrow(df)` and `DF.nrow(df_test)`:

In [7]:
df, df_test = partition(df, 0.7, rng=123)  # split 70:30 into df and df_test

println(DF.nrow(df))  # print number of rows on df
println(DF.nrow(df_test))  # print number of rows in df_test


624
267


## Splitting data into input features and target

In supervised learning, the target is the variable we want to predict, in this case `survived`. The other features will be inputs to our predictor. The following code puts the `df` column with name `survived` into the vector `y` (the target) and everything else, except `cabin`, which we're dropping, into a new dataframe called `X` (the input features).

In [8]:
y, X = unpack(df, ==(:survived), !=(:cabin))

(CategoricalArrays.CategoricalValue{String, UInt32}["1", "0", "0", "0", "1", "0", "0", "1", "1", "1"  …  "0", "1", "0", "0", "1", "0", "1", "0", "1", "0"], 624×6 DataFrame
 Row │ pclass  sex     age      sibsp  fare     embarked 
     │ Cat…    Cat…    Float64  Int64  Float64  Cat…?    
─────┼───────────────────────────────────────────────────
   1 │ 3       female     31.0      0   8.6833  S
   2 │ 3       male        2.0      4  29.125   Q
   3 │ 3       male       30.0      0   7.8958  S
   4 │ 2       male       34.0      1  26.0     S
   5 │ 1       female     54.0      1  78.2667  C
   6 │ 3       male       30.5      0   8.05    S
   7 │ 3       male       32.0      0   8.3625  S
   8 │ 3       male       32.0      0   8.05    S
  ⋮  │   ⋮       ⋮        ⋮       ⋮       ⋮        ⋮
 618 │ 3       male       36.0      0   0.0     S
 619 │ 2       male       42.0      0  13.0     S
 620 │ 3       male       26.0      0   7.8958  S
 621 │ 3       female     33.0      3  15.85    S
 

You can check `X` and `y` have the expected form by doing `schema(X)` and `scitype(y)`:

In [9]:
schema(X)

┌──────────┬───────────────────────────────┬────────────────────────────────────
│ names    │ scitypes                      │ types                             ⋯
├──────────┼───────────────────────────────┼────────────────────────────────────
│ pclass   │ Multiclass{3}                 │ CategoricalValue{String, UInt32}  ⋯
│ sex      │ Multiclass{2}                 │ CategoricalValue{String, UInt32}  ⋯
│ age      │ Continuous                    │ Float64                           ⋯
│ sibsp    │ Count                         │ Int64                             ⋯
│ fare     │ Continuous                    │ Float64                           ⋯
│ embarked │ Union{Missing, Multiclass{3}} │ Union{Missing, CategoricalValue{S ⋯
└──────────┴───────────────────────────────┴────────────────────────────────────
                                                                1 column omitted


In [10]:
scitype(y)

AbstractVector{Multiclass{2}} (alias for AbstractArray{Multiclass{2}, 1})

We'll want to do the same for the holdout test set:

In [11]:
y_test, X_test = unpack(df_test, ==(:survived), !=(:cabin))

(CategoricalArrays.CategoricalValue{String, UInt32}["0", "0", "1", "0", "0", "0", "1", "1", "0", "1"  …  "1", "0", "0", "0", "0", "1", "0", "0", "1", "0"], 267×6 DataFrame
 Row │ pclass  sex     age      sibsp  fare      embarked 
     │ Cat…    Cat…    Float64  Int64  Float64   Cat…?    
─────┼────────────────────────────────────────────────────
   1 │ 1       male       24.0      0   79.2     C
   2 │ 1       male       58.0      0   29.7     C
   3 │ 3       female     30.0      0    7.8292  Q
   4 │ 3       male       30.0      0    8.7125  C
   5 │ 2       male       39.0      0   13.0     S
   6 │ 3       female     30.0      0   15.2458  C
   7 │ 1       female     30.0      1  146.521   C
   8 │ 1       male       32.0      0   30.5     C
  ⋮  │   ⋮       ⋮        ⋮       ⋮       ⋮         ⋮
 261 │ 3       male       23.5      0    7.2292  C
 262 │ 3       male       28.0      0    7.8958  S
 263 │ 1       female     15.0      0  211.338   S
 264 │ 2       male       23.0      

## Choosing a supervised model

There are not many models that can directly handle missing values and a mixture of scitypes, as we have here. Here's how to list the ones that do:

In [12]:
models(matching(X,y))  # look for models that can be used for our dataset

4-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :abstract_type, :deep_properties, :docstring, :fit_data_scitype, :human_name, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :inverse_transform_scitype, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :predict_scitype, :prediction_type, :reporting_operations, :reports_feature_importances, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :transform_scitype, :input_scitype, :target_scitype, :output_scitype)}}:
 (name = ConstantClassifier, package_name = MLJModels, ... )
 (name = DecisionTreeClassifier, package_name = BetaML, ... )
 (name = DeterministicConstantClassifier, package_name = MLJModels, ... )
 (name = RandomForestClassifier, package_name = BetaML, ... )

This shortcoming can be addressed with data preprocessing provided by MLJ but not covered here, such as one-hot encoding and missing value imputation. We'll settle for the indicated decision tree.

The code for the decision tree model is not available until we explicitly load it, but we can already inspect it's documentation. Do this by entering `doc("DecisionTreeClassifier", pkg="BetaML")`. (To browse all MLJ model documentation use the Model Browser.)

An MLJ-specific method for loading the model code (and necessary packages) is shown below:

In [13]:
Tree = @load DecisionTreeClassifier pkg=BetaML  # Define a model
tree = Tree()  # assign it

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\luked\.julia\packages\MLJModels\EkXIe\src\loading.jl:159


import BetaML

 ✔


DecisionTreeClassifier(
  max_depth = 0, 
  min_gain = 0.0, 
  min_records = 2, 
  max_features = 0, 
  splitting_criterion = BetaML.Utils.gini, 
  rng = Random._GLOBAL_RNG())

The first line loads the model type, which we've called `Tree`; the second creates an object storing default hyperparameters for a `Tree` model.

We can specify different hyperparameters like this:

In [14]:
tree = Tree(max_depth=10)  # specify a hyperparameter

DecisionTreeClassifier(
  max_depth = 10, 
  min_gain = 0.0, 
  min_records = 2, 
  max_features = 0, 
  splitting_criterion = BetaML.Utils.gini, 
  rng = Random._GLOBAL_RNG())

## Training the model

We now bind the data to be used for training and the hyperparameter object `tree` we just created in a new object called a `machine`:

In [15]:
mach = machine(tree, X, y)

untrained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = 10, …)
  args: 
    1:	Source @582 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}, AbstractVector{Union{Missing, Multiclass{3}}}}}
    2:	Source @767 ⏎ AbstractVector{Multiclass{2}}


We train the model on all bound data by calling `fit!` on the machine. The exclamation mark `!` in `fit!` tells us that `fit!` mutates (changes) its argument. In this case the model's learned parameters (the actual decision tree) is stored in the `mach` object:

In [16]:
fit!(mach)

┌ Info: Training machine(DecisionTreeClassifier(max_depth = 10, …), …).
└ @ MLJBase C:\Users\luked\.julia\packages\MLJBase\fEiP2\src\machines.jl:492


trained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = 10, …)
  args: 
    1:	Source @582 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}, AbstractVector{Union{Missing, Multiclass{3}}}}}
    2:	Source @767 ⏎ AbstractVector{Multiclass{2}}


Before getting predictions for new inputs, let's start by looking at predictions for the inputs we trained on:

In [17]:
p = predict(mach, X)

624-element CategoricalDistributions.UnivariateFiniteVector{Multiclass{2}, String, UInt32, Float64}:
 UnivariateFinite{Multiclass{2}}(0=>0.0, 1=>1.0)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>0.0, 1=>1.0)
 UnivariateFinite{Multiclass{2}}(0=>0.915, 1=>0.0851)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>0.333, 1=>0.667)
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)
 UnivariateFinite{Multiclass{2}}(0=>0.0, 1=>1.0)
 ⋮
 UnivariateFinite{Multiclass{2}}(0=>0.0, 1=>1.0)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>0.5, 1=>0.5)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>0.25, 1=>0.75)
 UnivariateFinite{Multiclass{2}}(0=>1.0, 1=>0.0)
 UnivariateFinite{Multiclass{2}}(0=>0.444, 1=>0.556)

Notice that these are probabilistic predictions. For example, we have

In [18]:
p[6]

UnivariateFinite{Multiclass{2}}(0=>0.915, 1=>0.0851)

Extracting a raw probability requires an extra step. For example, to get this durvival probability (`1` corresponding to survival and `0` to death), we do this:

In [19]:
pdf(p[6], "1")

0.0851063829787234

We can also get "point" predictions using the `mode` function and Julia's broadcasting syntax:

In [20]:
yhat = mode.(p)
yhat[3:5]

3-element CategoricalArrays.CategoricalArray{String,1,UInt32}:
 "0"
 "0"
 "1"

## Evaluating model performance

Let's see how accurate our model is at predicting on the data it trained on:

In [21]:
accuracy(yhat, y)

0.9214743589743589

Over 90% accuracy! Better check the accuracy on the test data that the model hasn't seen:

In [22]:
yhat = mode.(predict(mach, X_test))
accuracy(yhat, y_test)

0.7790262172284644

Oh dear. We are most likely overfitting the model. Still, not a bad first step.

The evaluation we have just performed is known as holdout evaluation. MLJ provides tools for automating such evaluations, as well as more sophisticated ones such ass cross-validation.

## Tuning the model

Changing any hyperparameter of our model will alter it's performance. In particular, chainging certain parameters may mitigate overfitting.

In MLJ we can "wrap" the model to make it more automatically optimize a given hyperparameter, which it does by internally creating its own holdout set for evaluation (or using some other resampling scheme) and systematically searching over a specified range of one or more hyperparameters. Let's do that now for our decision tree.

First, we define a hyperparameter rang eover which we search:

In [23]:
r = range(tree, :max_depth, lower=0, upper=8)

NumericRange(0 ≤ max_depth ≤ 8; origin=4.0, unit=4.0)

Note that, according to the document string for the decision tree (`?Tree`) we see that `0` here means "no limit on `max_depth`".

Next, we apply MLJ's `TunedModel` wrapper to our tree, specifying the range and performance measure to use as a basis for optimization, as well as the resampling strategy we want to use, and the search method (grid in this case).

In [24]:
tuned_tree = TunedModel(
    tree,
    tuning=Grid(),
    range=r,
    measure=accuracy,
    resampling=Holdout(fraction_train=0.7),
)

ProbabilisticTunedModel(
  model = DecisionTreeClassifier(
        max_depth = 10, 
        min_gain = 0.0, 
        min_records = 2, 
        max_features = 0, 
        splitting_criterion = BetaML.Utils.gini, 
        rng = Random._GLOBAL_RNG()), 
  tuning = Grid(
        goal = nothing, 
        resolution = 10, 
        shuffle = true, 
        rng = Random._GLOBAL_RNG()), 
  resampling = Holdout(
        fraction_train = 0.7, 
        shuffle = false, 
        rng = Random._GLOBAL_RNG()), 
  measure = Accuracy(), 
  weights = nothing, 
  class_weights = nothing, 
  operation = nothing, 
  range = NumericRange(0 ≤ max_depth ≤ 8; origin=4.0, unit=4.0), 
  selection_heuristic = MLJTuning.NaiveSelection(nothing), 
  train_best = true, 
  repeats = 1, 
  n = nothing, 
  acceleration = CPU1{Nothing}(nothing), 
  acceleration_resampling = CPU1{Nothing}(nothing), 
  check_measure = true, 
  cache = true)

The new model `tuned_tree` behaves like the old, except the `max_depth` hyperparameter effectively becomes a learned parameter instead.

Training this `tuned_tree` actually performs two operations, under the hood:
 - Search for the best model using an internally constructed holdout set;
 - Retrain the "best" model on all available data

In [25]:
mach2 = machine(tuned_tree, X, y)
fit!(mach2)

┌ Info: Training machine(ProbabilisticTunedModel(model = DecisionTreeClassifier(max_depth = 10, …), …), …).
└ @ MLJBase C:\Users\luked\.julia\packages\MLJBase\fEiP2\src\machines.jl:492


┌ Info: Attempting to evaluate 9 models.
└ @ MLJTuning C:\Users\luked\.julia\packages\MLJTuning\nZnsJ\src\tuned_models.jl:727


Evaluating over 9 metamodels:  22%[=====>                   ]  ETA: 0:00:12

Evaluating over 9 metamodels: 100%[=========================] Time: 0:00:03


trained Machine; does not cache data
  model: ProbabilisticTunedModel(model = DecisionTreeClassifier(max_depth = 10, …), …)
  args: 
    1:	Source @378 ⏎ Table{Union{AbstractVector{Continuous}, AbstractVector{Count}, AbstractVector{Multiclass{3}}, AbstractVector{Multiclass{2}}, AbstractVector{Union{Missing, Multiclass{3}}}}}
    2:	Source @011 ⏎ AbstractVector{Multiclass{2}}


Here's how we can see what the optimal model actually is:

In [26]:
fitted_params(mach2).best_model

DecisionTreeClassifier(
  max_depth = 6, 
  min_gain = 0.0, 
  min_records = 2, 
  max_features = 0, 
  splitting_criterion = BetaML.Utils.gini, 
  rng = Random._GLOBAL_RNG())

Finally, let's test the self-tuning model on our existing holdout set:

In [27]:
yhat2 = mode.(predict(mach2, X_test))
accuracy(yhat2, y_test)

0.8164794007490637